<center>
    <h1 id='data-augmentation' style='color:#7159c1'>👁️‍🗨️ Data Augmentation 👁️‍🗨️</h1>
    <i>Making one Image Become N-Images</i>
</center>

---

The best way to improve the performance of a machine learning model is to train it on more data. The more examples the model has to learn from, the better it will be able to recognize which differences in images matter and which do not. More data helps the model to generalize better.

One easy way of getting more data is to use the data you already have. If we can transform the images in our dataset in ways that preserve the class, we can teach our classifier to ignore those kinds of transformations. For instance, whether a car is facing left or right in a photo doesn't change the fact that it is a Car and not a Truck. So, if we augment our training data with flipped images, our classifier will learn that "left or right" is a difference it should ignore.

And that's the whole idea behind data augmentation: add in some extra fake data that looks reasonably like the real data and your classifier will improve!!

---

Typically, many kinds of transformation are used when augmenting a dataset. These might include rotating the image, adjusting the color or contrast, warping the image, or many other things, usually applied in combination. Here is a sample of the different ways a single image might be transformed.

It's important to remember though that not every transformation will be useful on a given problem. Most importantly, whatever transformations you use should not mix up the classes. If you were training a digit recognizer, for instance, rotating images would mix up '9's and '6's. In the end, the best approach for finding good augmentations is the same as with most ML problems: try it and see!

In [1]:
# ---- Importations ----
import os, warnings # pip install os warnings
import matplotlib.pyplot as plt # pip install matplotlibb
from matplotlib import gridspec
import mplcyberpunk # pip install mplcyberpunk
import pandas as pd # pip install pandas

import numpy as np # pip install numpy
import tensorflow as tf # pip install tensorflow
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# ---- Reproducability ----
def set_seed(seed=20242401):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
set_seed()

# ---- Setting Matplotlib Defaults ----
plt.rc('figure', autolayout=True)
plt.rc(
    'axes'
    , labelweight='bold'
    , labelsize='large'
    , titleweight='bold'
    , titlesize=18
    , titlepad=10
)
plt.rc('image', cmap='magma')
plt.style.use('cyberpunk')
warnings.filterwarnings("ignore")

In [2]:
# ---- Preparing Datasets ----
train_df = image_dataset_from_directory(
    './datasets/car-or-truck/train',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

valid_df = image_dataset_from_directory(
    './datasets/car-or-truck/valid',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)

# ---- Pipeline ----
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_df = (
    train_df
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

valid_df = (
    valid_df
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

Found 5117 files belonging to 2 classes.
Found 5051 files belonging to 2 classes.


In [3]:
# ---- Loading Pretrained Base ----
pretrained_base = tf.keras.models.load_model(
    './models/vgg16-pretrained-base',
)
pretrained_base.trainable = False

# ---- Creating the Head Model ----
model_1 = keras.Sequential([
    # Input Layer #
    layers.InputLayer(input_shape=[128, 128, 3]),

    # Preprocessing
    preprocessing.RandomFlip('horizontal'), # flip left-to-right
    preprocessing.RandomContrast(0.5), # contrast change by up to 50%

    # Base
    pretrained_base,

    # Head
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

model_2 = keras.Sequential([

	# Input Layer #
    layers.InputLayer(input_shape=[128, 128, 3]),
    
    # Data Augmentation #
    preprocessing.RandomContrast(factor=0.10),
    preprocessing.RandomFlip(mode='horizontal'),
    preprocessing.RandomRotation(factor=0.10),

    ## Base ##

    # Block One #
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Block Two #
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    # Block Three #
    layers.BatchNormalization(renorm=True),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPool2D(),

    ## Head ##
    layers.BatchNormalization(renorm=True),
    layers.Flatten(),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid'),
])

In [ ]:
# ---- Training ----
model_2.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

history = model_2.fit(
    train_df,
    validation_data=valid_df,
    epochs=2,
    # epochs=30,
    verbose=0,
)

In [ ]:
# ---- Evaluating - Loss ----
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
plt.show()

In [ ]:
# ---- Evaluating - Binary Accuracy ----
history_frame.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot();
plt.show()

---

In [ ]:
# ---- Some Augmentation Functions That We Can Apply ----
preprocessing.RandomContrast(factor=0.5),
preprocessing.RandomFlip(mode='horizontal'), # meaning, left-to-right
preprocessing.RandomFlip(mode='vertical'), # meaning, top-to-bottom
preprocessing.RandomWidth(factor=0.15), # horizontal stretch
preprocessing.RandomRotation(factor=0.20),
preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),

---

<h1 id='reach-me' style='color:#7159c1; border-bottom:3px solid #7159c1; letter-spacing:2px; font-family:JetBrains Mono; font-weight: bold; text-align:left; font-size:240%;padding:0'>📫 | Reach Me</h1>

> **Email** - [csfelix08@gmail.com](mailto:csfelix08@gmail.com?)

> **Linkedin** - [linkedin.com/in/csfelix/](https://www.linkedin.com/in/csfelix/)

> **GitHub:** - [CSFelix](https://github.com/CSFelix)

> **Kaggle** - [DSFelix](https://www.kaggle.com/dsfelix)

> **Portfolio** - [CSFelix.io](https://csfelix.github.io/).